## Importing the Packages

In [1]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima
from pmdarima.arima import OCSBTest 
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model
import seaborn as sns
import yfinance
import warnings
warnings.filterwarnings("ignore")
sns.set()

## Importing the Data

In [2]:
df = yfinance.download(tickers='VOW3.DE,PAH3.DE,BMW.DE',interval='1d',group_by='ticker',auto_adjust=True,treads=True)

[*********************100%***********************]  3 of 3 completed


In [3]:
df1 = df.copy()

In [4]:
df1.head()

PAH3.DE                       VOW3.DE                        \
              Open High Low Close Volume    Open High Low Close Volume   
Date                                                                     
1996-11-08     NaN  NaN NaN   NaN    NaN     NaN  NaN NaN   NaN    NaN   
1996-11-11     NaN  NaN NaN   NaN    NaN     NaN  NaN NaN   NaN    NaN   
1996-11-12     NaN  NaN NaN   NaN    NaN     NaN  NaN NaN   NaN    NaN   
1996-11-13     NaN  NaN NaN   NaN    NaN     NaN  NaN NaN   NaN    NaN   
1996-11-14     NaN  NaN NaN   NaN    NaN     NaN  NaN NaN   NaN    NaN   

              BMW.DE                                         
                Open      High       Low     Close   Volume  
Date                                                         
1996-11-08  7.883176  7.883176  7.866293  7.866293   767000  
1996-11-11  7.874518  7.878847  7.827765  7.845080   260000  
1996-11-12  7.861965  7.934259  7.832094  7.904390  1066000  
1996-11-13  7.941185  7.941185  7.874519  7.891834   793000  
1996-11-14  7.881012  7.917376  7.849410  7.917376   351000

In [6]:
df1.tail()

PAH3.DE                                                VOW3.DE  \
                 Open       High        Low      Close    Volume        Open   
Date                                                                           
2020-05-25  47.189999  47.400002  46.200001  47.400002  245297.0  134.500000   
2020-05-26  47.959999  49.520000  47.669998  49.240002  747063.0  135.000000   
2020-05-27  49.380001  52.299999  49.299999  50.799999  993371.0  137.240005   
2020-05-28  51.720001  52.139999  49.020000  50.779999  714475.0  142.600006   
2020-05-29  49.220001  50.160000  48.509998  49.490002  117764.0  134.460007   

                                                              BMW.DE  \
                  High         Low       Close     Volume       Open   
Date                                                                   
2020-05-25  134.500000  131.160004  133.960007  1071253.0  50.759998   
2020-05-26  137.800003  134.000000  136.839996  1489657.0  50.740002   
2020-05-27  146.699997  136.979996  140.119995  2983830.0  52.000000   
2020-05-28  142.779999  133.940002  137.220001  2347733.0  55.250000   
2020-05-29  134.619995  131.380005  133.100006   334698.0  52.740002   

                                                      
                 High        Low      Close   Volume  
Date                                                  
2020-05-25  50.790001  49.080002  50.290001  1796645  
2020-05-26  51.869999  50.099998  51.869999  2720565  
2020-05-27  55.810001  51.779999  54.389999  4266200  
2020-05-28  55.369999  53.220001  54.000000  2340231  
2020-05-29  53.500000  52.279999  52.660000   431838

In [7]:
df1.isna().sum()

PAH3.DE  Open      3126
         High      3126
         Low       3126
         Close     3126
         Volume    3126
VOW3.DE  Open       458
         High       458
         Low        458
         Close      458
         Volume     458
BMW.DE   Open         0
         High         0
         Low          0
         Close        0
         Volume       0
dtype: int64

In [5]:
df1.describe()

PAH3.DE                                                       \
              Open         High          Low        Close        Volume   
count  2875.000000  2875.000000  2875.000000  2875.000000  2.875000e+03   
mean     42.843989    43.414083    42.199775    42.818792  6.194769e+05   
std      12.451239    12.452437    12.419621    12.439119  4.579764e+05   
min      17.316915    17.637715    16.111004    17.132750  0.000000e+00   
25%      31.758690    32.434797    31.110412    31.847635  3.642390e+05   
50%      43.511019    44.025743    42.811839    43.424850  4.941330e+05   
75%      52.666274    53.083817    51.958080    52.463947  7.178730e+05   
max      70.941258    72.015084    70.689935    71.588600  5.944992e+06   

           VOW3.DE                                                       \
              Open         High          Low        Close        Volume   
count  5543.000000  5543.000000  5543.000000  5543.000000  5.543000e+03   
mean     77.868752    78.878407    76.721249    77.835895  8.430495e+05   
std      57.864499    58.430221    57.212971    57.804104  1.025618e+06   
min      12.239773    12.697073    11.912363    11.997676  0.000000e+00   
25%      19.652698    19.960238    19.253579    19.665441  2.082995e+05   
50%      64.531634    65.358843    63.520634    64.607224  6.319880e+05   
75%     131.943772   133.622979   130.412865   131.835373  1.144462e+06   
max     231.188500   233.816665   222.324050   227.357635  1.753497e+07   

            BMW.DE                                                       
              Open         High          Low        Close        Volume  
count  6001.000000  6001.000000  6001.000000  6001.000000  6.001000e+03  
mean     35.486772    35.901838    35.023051    35.469209  2.283369e+06  
std      23.098648    23.280338    22.876988    23.084104  1.518630e+06  
min       7.861965     7.878847     7.704819     7.832092  0.000000e+00  
25%      16.908336    17.149755    16.632563    16.908337  1.352701e+06  
50%      23.602203    23.756265    23.233684    23.534452  1.978459e+06  
75%      60.154493    60.655534    59.436845    60.005390  2.805392e+06  
max      95.009579    95.356327    92.736440    94.470192  2.871954e+07

## Defining Key Dates

In [8]:
# Starting Date
start_date = '2009-04-05'

# First Official Announcement - 49.9% - volkswagen bought 49.9% in porche
ann_1 = '2009-12-09'
# Second Official Announcement - 51.1% - announced remaining percent buyout
ann_2 = '2012-07-05'
#Ending Date
end_date = '2014-01-01'## Any value
# Dieselgate
d_gate='2015-09-20' ##date of scam

## Pre-processing the Data

In [3]:
# Extracting Closing Prices

# Creating Returns


# Creating Squared Returns

# Extracting Volume


In [4]:
# Assigning the Frequency and Filling NA Values


In [5]:
# Removing Surplus Data


## Plotting the Prices

In [6]:
#color = "#33B8FF"
#color = "#49FF3A"
#color = "#FEB628"

#color = "#1E7EB2"
#color = "#2FAB25"
#color = "#BA861F"

#color = "#0E3A52"
#color = "#225414"
#color = "#7C5913"


## Correlation

## Best Fitting Models

### For Volkswagen

### For Porsche

## Predictions for the Future

### For Porsche

### For Volkswagen

## Volatility

### Volatility of VW for Each Period

### Volatility Trends for Each Period